BM25

In [1]:
!pip install -U beir
!pip install 'elasticsearch<7.14.0'

  Using cached beir-0.2.2-py3-none-any.whl (49 kB)
  Using cached pytrec_eval-0.5.tar.gz (15 kB)
  Using cached faiss_cpu-1.7.1.post2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.4 MB)
  Using cached sentence-transformers-2.1.0.tar.gz (78 kB)
  Using cached tensorflow_text-2.6.0-cp37-cp37m-manylinux1_x86_64.whl (4.4 MB)
     |████████████████████████████████| 2.9 MB 6.7 MB/s 
     |████████████████████████████████| 3.3 MB 59.4 MB/s 
     |████████████████████████████████| 1.2 MB 57.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
     |████████████████████████████████| 636 kB 53.7 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=264857 sha256=3d70a5c5a0ee7f9cb08eed9144a391014de4d9e0d1838c7c798720917e6240fd
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
  Created wheel for sentence-transformer

In [2]:
import pathlib, os
import time
import pandas as pd
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
if not os.path.exists('elasticsearch-oss-7.9.2-linux-x86_64.tar.gz'):
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
  !tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !sudo chown -R daemon:daemon elasticsearch-7.9.2/
  !shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

--2021-10-09 20:27:05--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229941304 (219M) [application/x-gzip]
Saving to: ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’

elasticsearch-oss-7 100%[===================>] 219.29M  22.6MB/s    in 15s     

2021-10-09 20:27:20 (14.7 MB/s) - ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’ saved [229941304/229941304]

--2021-10-09 20:27:20--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 20

In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [5]:
time.sleep(20)

In [6]:
!ps -ef | grep elasticsearch

root         453     451  0 20:27 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       454     453 99 20:27 ?        00:00:20 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-14599886008618856690 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

In [7]:
!curl -sX GET "localhost:9200/"

{
  "name" : "d0b667c9c51d",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "rAJ_2nVKSoGZJcDUm2mWSw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [8]:
def eval_metrics(model_name, ndcg, _map, recall, precision):
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [9]:
hostname = 'localhost' 
url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'
out_dir = os.path.join(os.getcwd(), 'datasets')

In [10]:
dataset_sf = 'scifact'
index_sf = dataset_sf
data_path_sf = util.download_and_unzip(url.format(dataset_sf), out_dir)
corpus_sf, queries_sf, qrels_sf = GenericDataLoader(data_path_sf).load(split='test') # 'test', 'train', 'dev'
model_sf = BM25(index_name=index_sf, hostname=hostname, initialize=True) # initialize=True : reindex
retriever_sf = EvaluateRetrieval(model_sf)
results_sf = retriever_sf.retrieve(corpus_sf, queries_sf)
ndcg_sf, _map_sf, recall_sf, precision_sf = retriever_sf.evaluate(qrels_sf, results_sf, retriever_sf.k_values)

/content/datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

  0%|          | 0/5183 [00:00<?, ?it/s]

que: 100%|██████████| 3/3 [00:20<00:00,  6.98s/it]


In [11]:
dataset_sd = 'scidocs'
index_sd = dataset_sd
data_path_sd = util.download_and_unzip(url.format(dataset_sd), out_dir)
corpus_sd, queries_sd, qrels_sd = GenericDataLoader(data_path_sd).load(split='test')
model_sd = BM25(index_name=index_sd, hostname=hostname, initialize=True)
retriever_sd = EvaluateRetrieval(model_sd)
results_sd = retriever_sd.retrieve(corpus_sd, queries_sd)
ndcg_sd, _map_sd, recall_sd, precision_sd = retriever_sd.evaluate(qrels_sd, results_sd, retriever_sd.k_values)

/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

que: 100%|██████████| 8/8 [00:45<00:00,  5.70s/it]


In [12]:
dataset_fv = 'fever'
index_fv = dataset_fv
data_path_fv = util.download_and_unzip(url.format(dataset_fv), out_dir)
corpus_fv, queries_fv, qrels_fv = GenericDataLoader(data_path_fv).load(split='test')
model_fv = BM25(index_name=index_fv, hostname=hostname, initialize=True)
retriever_fv = EvaluateRetrieval(model_fv)
results_fv = retriever_fv.retrieve(corpus_fv, queries_fv)
ndcg_fv, _map_fv, recall_fv, precision_fv = retriever_fv.evaluate(qrels_fv, results_fv, retriever_fv.k_values)

/content/datasets/fever.zip:   0%|          | 0.00/1.15G [00:00<?, ?iB/s]

  0%|          | 0/5416568 [00:00<?, ?it/s]

que: 100%|██████████| 53/53 [21:09<00:00, 23.95s/it]


In [13]:
eval_metrics('SCIFACT', ndcg_sf, _map_sf, recall_sf, precision_sf)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
SCIFACT,0.64619,0.65173,0.65202,0.69254,0.71588,0.72322,0.09067,0.01043,0.00111,0.81978,0.92256,0.98


In [14]:
eval_metrics('SCIDOCS', ndcg_sd, _map_sd, recall_sd, precision_sd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
SCIDOCS,0.09641,0.11193,0.11418,0.16478,0.23043,0.27381,0.0858,0.0181,0.00285,0.17392,0.36757,0.57872


In [15]:
eval_metrics('FEVER', ndcg_fv, _map_fv, recall_fv, precision_fv)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
FEVER,0.58538,0.5909,0.59115,0.64938,0.67394,0.67935,0.08885,0.01029,0.00109,0.81413,0.92151,0.95976


end of fun.